In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [13]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# الاحتفاظ بـ PassengerId من بيانات الاختبار
test_passenger_ids = test_df["PassengerId"]

# 2. دمج البيانات مؤقتاً لتنظيفها بنفس الطريقة
full_df = pd.concat([train_df, test_df], sort=False)

# 3. معالجة القيم المفقودة
full_df["Age"].fillna(full_df["Age"].median(), inplace=True)
full_df["Fare"].fillna(full_df["Fare"].median(), inplace=True)
full_df["Embarked"].fillna(full_df["Embarked"].mode()[0], inplace=True)

# 4. تحويل الأعمدة النصية إلى رقمية
label_cols = ["Sex", "Embarked"]
for col in label_cols:
    le = LabelEncoder()
    full_df[col] = le.fit_transform(full_df[col])

# 5. استخراج ميزات إضافية (اختياري لتحسين الأداء)
full_df["FamilySize"] = full_df["SibSp"] + full_df["Parch"] + 1
full_df["IsAlone"] = (full_df["FamilySize"] == 1).astype(int)

# 6. إسقاط الأعمدة غير المفيدة
drop_cols = ["PassengerId", "Name", "Ticket", "Cabin"]
full_df.drop(columns=drop_cols, inplace=True)

# 7. فصل البيانات مرة أخرى
train_clean = full_df.iloc[:len(train_df), :]
test_clean = full_df.iloc[len(train_df):, :]

X = train_clean.drop("Survived", axis=1)
y = train_df["Survived"]

# 8. تقسيم البيانات
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

/tmp/ipykernel_36/1593353898.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_df["Age"].fillna(full_df["Age"].median(), inplace=True)
/tmp/ipykernel_36/1593353898.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

In [18]:
from sklearn.metrics import accuracy_score

# 9. إعداد نموذج XGBoost
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)

# 10. تحديد نطاق البحث عن المعلمات
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "min_child_weight": [1, 3, 5]
}

# 11. البحث عن أفضل المعلمات
grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# 12. عرض أفضل المعلمات
print("أفضل المعلمات:", grid_search.best_params_)
print("أفضل دقة:", grid_search.best_score_)

# 13. التقييم على مجموعة التحقق
best_model = grid_search.best_estimator_
y_pred_val = best_model.predict(X_val)
print("دقة على مجموعة التحقق:", accuracy_score(y_val, y_pred_val))


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
أفضل المعلمات: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}
أفضل دقة: 0.8384713877671623
دقة على مجموعة التحقق: 0.8212290502793296


In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
# تقسيم البيانات للتقييم الداخلي
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# تعريف موديل XGBoost بأفضل المعلمات
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=1.0,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

# تدريب الموديل
xgb_model.fit(X_train, y_train)

# التنبؤ على مجموعة التحقق
y_val_pred = xgb_model.predict(X_val)

# تقييم الأداء
print("Accuracy on validation set:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))


Accuracy on validation set: 0.8212290502793296
              precision    recall  f1-score   support

           0       0.82      0.90      0.85       105
           1       0.83      0.72      0.77        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



In [21]:

# الاحتفاظ بـ PassengerId للملف النهائي
test_passenger_ids = test_df["PassengerId"]

# 2. دمج البيانات مؤقتاً للتنظيف
full_df = pd.concat([train_df, test_df], sort=False)

# 3. معالجة القيم المفقودة
full_df["Age"].fillna(full_df["Age"].median(), inplace=True)
full_df["Fare"].fillna(full_df["Fare"].median(), inplace=True)
full_df["Embarked"].fillna(full_df["Embarked"].mode()[0], inplace=True)

# 4. تحويل الأعمدة النصية إلى رقمية
for col in ["Sex", "Embarked"]:
    le = LabelEncoder()
    full_df[col] = le.fit_transform(full_df[col])

# 5. استخراج ميزات إضافية
full_df["FamilySize"] = full_df["SibSp"] + full_df["Parch"] + 1
full_df["IsAlone"] = (full_df["FamilySize"] == 1).astype(int)

# 6. إزالة الأعمدة غير المفيدة
full_df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

# 7. فصل البيانات مرة أخرى
train_clean = full_df.iloc[:len(train_df), :]
test_clean = full_df.iloc[len(train_df):, :]

# 8. إعداد X و y
X = train_clean.drop("Survived", axis=1)
y = train_clean["Survived"]

# 9. تعريف أفضل موديل XGBoost بناءً على GridSearchCV السابق
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=1.0,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

# 10. تدريب الموديل على كامل بيانات التدريب
xgb_model.fit(X, y)

# 11. تجهيز بيانات الاختبار
X_test_final = test_clean.drop("Survived", axis=1, errors='ignore')

# 12. التنبؤ على بيانات الاختبار
y_test_pred = xgb_model.predict(X_test_final)

# 13. إنشاء ملف submission
submission = pd.DataFrame({
    "PassengerId": test_passenger_ids,
    "Survived": y_test_pred
})

submission.to_csv("titanic_submission2.csv", index=False)
print("Submission file created!")

Submission file created!


/tmp/ipykernel_36/2941229200.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_df["Age"].fillna(full_df["Age"].median(), inplace=True)
/tmp/ipykernel_36/2941229200.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True